# **Normalizzazione dei Nomi Aziendali**

## **Introduzione**

Nel processo di deduplicazione e record linkage, è essenziale normalizzare i nomi aziendali per evitare variazioni dovute a abbreviazioni, formati o errori di battitura. In questa fase, implementiamo una strategia per rendere i nomi uniformi e comparabili.

---

## **Step 1: Importazione delle Librerie e del DataFrame**

In [2]:
import re
import pandas as pd
import unidecode

AZIENDE_CSV = '../aziende.csv'
IMPIEGATI_CSV = '../impiegati.csv'
companies_df = pd.read_csv(AZIENDE_CSV)

C:\Users\emigi\AppData\Local\Temp\ipykernel_27024\3147525914.py:7: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  companies_df = pd.read_csv(AZIENDE_CSV)


---

## **Step 2: Definizione della Mappa di Sostituzione**
Utilizziamo un dizionario per standardizzare suffissi comuni nei nomi aziendali.
### **Questo va sicuramente migliorato magari passando tutta la prima colonna a chat**

In [3]:
REPLACEMENTS = {
    r"&": "",
    r"(?<=\s|,|\.)inc\.?\b": "",
    r"(?<=\s|,|\.)ltd\b": "",
    r"(?<=\s|,|\.)llc\b": "",
    r"(?<=\s|,|\.)plc\b": "",
    r"(?<=\s|,|\.)co\.?\b": "",  # Handles "co." and "co,"
    r"(?<=\s|,|\.)corp\.?\b": "",
    r"(?<=\s|,|\.)srl\b": "",
    r"(?<=\s|,|\.)spa\b": "",
    r"(?<=\s|,|\.)ag\b": "",
    r"(?<=\s|,|\.)sa\b": "",
    r"(?<=\s|,|\.)ab\b": "",
    r"(?<=\s|,|\.)ou\b": "",
    r"(?<=\s|,|\.)as\b": "",
    r"(?<=\s|,|\.)gmbh\b": "",
    r"(?<=\s|,|\.)holding\b": "",
    r"(?<=\s|,|\.)holdings\b": "",
    r"(?<=\s|,|\.)corporation\b": "",
    r"(?<=\s|,|\.)industries\b": "",
    r"(?<=\s|,|\.)company limited\b": "",
    r"(?<=\s|,|\.)technology\b": "",
    r"(?<=\s|,|\.)solutions\b": "",
    r"(?<=\s|,|\.)systems\b": "",
    r"(?<=\s|,|\.)services\b": "",
    r"(?<=\s|,|\.)public company\b": "",

    # New patterns to remove dotted abbreviations
    r"\bs\.?p\.?a\.?\b": "",  # Handles "S.p.A." and variations
    r"\bs\.?r\.?l\.?\b": "",  # Handles "S.R.L."
    r"\bs\.?a\.?\b": "",      # Handles "S.A."
}

---

## **Step 3: Funzione di Normalizzazione**

Questa funzione applica una serie di trasformazioni per ottenere nomi uniformi:

- Conversione in minuscolo
- Rimozione di accenti
- Eliminazione di caratteri speciali
- Sostituzione dei suffissi tramite `REPLACEMENTS`

In [4]:
def normalize_name(name):
    """Normalizza i nomi delle aziende."""
    if pd.isna(name) or not isinstance(name, str):
        return ""
    
    # 1. Rimuove accenti e caratteri speciali
    name = unidecode.unidecode(name)
    
    # 2. Minuscolo e rimozione spazi extra
    name = name.lower().strip()
    
    # 3. Rimuove testo tra virgolette
    name = re.sub(r'"([^"]+)"','', name) # 3. Rimuove testo tra virgolette
    
    # 4. Applica le sostituzioni dalla mappa REPLACEMENTS
    for pattern, replacement in REPLACEMENTS.items():
        name = re.sub(pattern, replacement, name)
    
    # 5. Pulisce caratteri inutili
    name = re.sub(r"\s*,+\s*", " ", name)
    name = re.sub(r"\s*\.+\s*", " ", name) 
    
    # 6. Sostituisce spazi multipli con uno solo
    name = re.sub(r'\s+', ' ', name)  
    
    # 7. Eliminare punteggiatura indesiderata alla fine 
    name = re.sub(r"[.,\-]+$", "", name)
    
    return name.strip()

---

## **Step 4: Applicazione alla Colonna 'company_name'**

Eseguiamo la normalizzazione sulla colonna contenente i nomi aziendali.

In [5]:
companies_df['normalized_name'] = companies_df['company_name'].apply(normalize_name)
companies_df['modified'] = companies_df['normalized_name'] != companies_df['company_name']

---

## **Step 5: Visualizzazione e salvataggio del Risultato**

Mostriamo i primi risultati per verificare la trasformazione
e salva tutto in un csv

In [6]:
num_modified = companies_df['modified'].sum()
total_entries = len(companies_df)
percentage_modified = (num_modified / total_entries) * 100

print(f"Entries modificate: {num_modified} su {total_entries} ({percentage_modified:.2f}%)")

modified_entries = companies_df[companies_df['modified']][['company_name', 'normalized_name']]
display(modified_entries.sample(25))

companies_df = companies_df.sort_values(by=['normalized_name'])
companies_df[['company_name', 'normalized_name', 'modified']].to_csv('normalized_names.csv', index=False)

Entries modificate: 37656 su 76808 (49.03%)


,company_name,normalized_name
11869,calpeda spa,calpeda
41519,melon sa,melon
17479,daikin industries,daikin
74081,xpo inc,xpo
69072,u.t. communications spa,u t communications
46842,oeneo sa,oeneo
34176,"itochu enex co.,ltd.",itochu enex
48371,p & a projects limited,p a projects limited
46727,"obrascón huarte lain, sa",obrascon huarte lain
38869,lion asiapac ltd,lion asiapac


---

## **Conclusioni**

Questo approccio garantisce una normalizzazione efficace dei nomi aziendali, riducendo le variazioni dovute a differenze di scrittura, abbreviazioni e formati. È facilmente estendibile aggiornando la mappa `REPLACEMENTS` con nuove regole di sostituzione.



In [7]:
companies_df['company_name'] = companies_df['normalized_name']
companies_df = companies_df.sort_values(by=['company_name'])
companies_df.to_csv('../aziende_normalizzate.csv', index=False)

# **Normalizzazione delle città**

In [8]:
filtered_df = companies_df[companies_df['city'].notna() & (companies_df['city'].str.strip() != "")].copy()
display(filtered_df[['company_name', 'city']].sample(25))

,company_name,city
49870,pharmavite,west hills
28957,h2o ai,mountain view
23086,feralpi siderurgica,lonato del garda
37919,lannutti,cuneo
5586,ashurst llp,london
39060,lnrs data limited,sutton
57490,scholastic,new york
25931,gemini,new york
46590,nxin,beijing
21938,estonian cell,"kunda, viru-nigula vald"


### **Step 1: Eliminare eventuale testo tra parentesi** 
La cosa che sempra più intuitiva da fare è eliminare eventuale testo tra parentesi dopo la città (perdendo un po' di informazione, ma comunque accettabile)

In [9]:
def clean_city(city):
    if pd.isna(city) or not isinstance(city, str):
        return ""
    
    city = re.sub(r"\s*\(.*?\)\s*", "", city).strip()
    return city 

In [10]:
filtered_df['city'] = filtered_df['city'].apply(clean_city)
display(filtered_df[['company_name', 'city']].sample(25))

,company_name,city
29335,haomao ai,beijing
16000,conserve italia - consorzio italiano fra coope...,san lazzaro di savena
54927,retal pa,donora
11000,bridgepoint advisers,london
19310,dowding and mills limited,birmingham
62443,state street global advisors trust company,boston
14382,christian dior uk limited,london
34193,itron international,liberty lake
4209,anaplan,san francisco
10598,boohoo group,manchester


In [11]:
# Apply the changes to the df
companies_df['city'] = companies_df['city'].apply(clean_city)

### **Step 2: Tradurre nomi di città italiane in italiano** 

Bisognerebbe anche indagare se qualche città viene riportata in lingue diverse...

In [12]:
# Query in italiano
milano_df = filtered_df[filtered_df["city"] == "milano"]
roma_df = filtered_df[filtered_df["city"] == "roma"]

# Query in inglese
milan_df = filtered_df[filtered_df["city"] == "milan"]
rome_df = filtered_df[filtered_df["city"] == "rome"]

In [13]:
display(milano_df[['company_name', 'city']])
display(milan_df[['company_name', 'city']])

,company_name,city
562,aberdeen standard investments ireland limited,milano
622,above comparison,milano
623,above comparison,milano
766,accenture,milano
765,accenture,milano
...,...,...
73454,withsecure,milano
74336,yes ticket,milano
74335,yes ticket,milano
75694,zurich italy bank,milano


,company_name,city
57140,satispay,milan
57364,scalapay,milan


In [14]:
display(roma_df[['company_name', 'city']])
display(rome_df[['company_name', 'city']])

,company_name,city
1431,aec underwriting agenzia di assicurazione e ri...,roma
1430,aec underwriting agenzia di assicurazione e ri...,roma
1739,agenzia delle entrate,roma
2463,ald automotive italia,roma
4282,angelini pharma italia aziende chimiche riunit...,roma
...,...,...
68023,trenitalia,roma
69746,universita' degli studi di roma la sapienza,roma
69748,universita' degli studi roma tre,roma
73045,western union payment ireland limited,roma


,company_name,city


Probabilmente, visto la sorgente dati, le aziende italiane sono state estratte da repository italiani o almeno convertite con città in italiano. Solo due aziende (*satispay* e *scalapay*) hanno il nome riportato in inglese.

Le città italiane sono segnate maggiormente in italiano possiamo pensare di **convertire le città con nome inglese in italiano** 

Abbiamo bisogno di un dizionario che mappi i nomi delle città (City-mapping). Evitiamo di scriverlo da zero e ne usiamo uno noto da qualche libreria (*do not reinvent the wheel...*).

Usiamo dunque la libreria `geopy` che con **GeoNames**, un database globale di località, ci permette di convertire i nomi 

In [15]:
from geopy.geocoders import Nominatim
import time

geolocator = Nominatim(user_agent="city_normalizer")
city_cache = {}

In [16]:
def get_city_in_italian(city):
    if pd.isna(city) or not isinstance(city, str) or city == "":
        return ""

    if city in city_cache:
        print(f"Cache hit! {city}")
        return city_cache[city]
    
    try:
        location = geolocator.geocode(city, exactly_one=True, timeout=10)
        if location and "Italia" in location.address:
            location_it = geolocator.geocode(city, language="it", timeout=10)
            if location_it:
                print(f"translating {location_it}...")
                it_city = location_it.raw.get("display_name", city).split(",")[0].lower().strip()
                city_cache[city] = it_city
                return it_city
    except Exception as e:
        print(f"Errore con {city}: {e}")

    city_cache[city] = city
    return city

In [17]:
def apply_geocoding(city):
    result = get_city_in_italian(city)
    time.sleep(1)
    return result

In [18]:
milan_df['city'] = milan_df['city'].apply(apply_geocoding)
display(milan_df[['company_name', 'city']])

translating Milano, Lombardia, Italia...
Cache hit! milan


C:\Users\emigi\AppData\Local\Temp\ipykernel_27024\1402383521.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  milan_df['city'] = milan_df['city'].apply(apply_geocoding)


,company_name,city
57140,satispay,milano
57364,scalapay,milano


**GeoNames** funziona, adesso applichiamolo all'intero dataframe

In [19]:
companies_df['city_translate'] = companies_df.apply(
    lambda row: apply_geocoding(row['city']), 
    axis=1
)
companies_df['modified'] = companies_df['city'] != companies_df['city_translate']

KeyboardInterrupt: 

In [58]:
num_modified = companies_df['modified'].sum()
total_cities = len(companies_df)
percentage_modified = (num_modified / total_cities) * 100

print(f"Città tradotte: {num_modified} su {total_cities} ({percentage_modified:.2f}%)")

translated_cities = companies_df[companies_df['modified']][['company_name', 'city', 'city_translate']]
display(translated_cities.sample(25))

companies_df = companies_df.sort_values(by=['company_name'])
companies_df[['company_name', 'city', 'city_translate']].to_csv('translated_cities.csv', index=False)

Città tradotte: 37656 su 76808 (49.03%)


KeyError: "['city_translate'] not in index"

NORMALIZZAZIONE CAMPO COUNTRY

In [71]:
filtered_df = companies_df[companies_df['country'].notna() & (companies_df['country'].str.strip() != "")].copy()
display(filtered_df[['company_name', 'country']].sample(25))

,company_name,country
67110,tkh group,netherlands
54500,reflow hub,australia
53122,qburst,india
60483,sneakers jackets,france
21314,enhabit,usa
62899,subsea 7 (uk service company) limited,united kingdom
26972,gong galaxy,france
1132,adenes italia,italy
38667,life healthcare group,south africa
67320,tomy company,japan


definizione di un mapping e regex per eliminare sostituire valori in modo corretto nel campo country, ad esempio abbreviazioni e altri casi particolari

In [21]:
import re
import pandas as pd

def clean_country(country): 
    if pd.isna(country) or not isinstance(country, str):
        return ""
    
    # Rimuove parentesi tonde ( ), quadre [ ] e singole virgolette '
    country = re.sub(r"[\[\]']", "", country)  
    country = re.sub(r"\s*\(.*?\)\s*", "", country).strip()  
    
    return country


In [35]:
filtered_df['country'] = filtered_df['country'].apply(clean_country)
display(filtered_df[['company_name', 'country']].sample(25))

,company_name,country
20394,egomnia,italy
36158,keltbray built environment limited,united kingdom
7695,banca nazionale del lavoro,it09339391006
32804,infocom,japan
30971,hotglue,australia
17419,daewoong pharmaceutical,south korea
54793,rent the runway,usa
10976,brf,brazil
5940,at s austria technologie systemtechnik,austria
31259,hudco,india


In [23]:
country_mapping = {
    'it': 'italy',            # Codici e abbreviazioni per l'Italia
    'uk': 'united kingdom',   # Abbreviazione per il Regno Unito
    'usa': 'united states',   # Abbreviazione per gli Stati Uniti
    'us': 'united states',    # Abbreviazione per gli Stati Uniti (altro formato)
    'fr': 'france',           # Abbreviazione per la Francia
    'de': 'germany',          # Abbreviazione per la Germania
    'es': 'spain',            # Abbreviazione per la Spagna
    'ca': 'canada',           # Abbreviazione per il Canada
    'au': 'australia',        # Abbreviazione per l'Australia
    'cn': 'china',            # Abbreviazione per la Cina
    'in': 'india',            # Abbreviazione per l'India
    'jp': 'japan',            # Abbreviazione per il Giappone
    'br': 'brazil',           # Abbreviazione per il Brasile
    'kr': 'south korea',      # Abbreviazione per la Corea del Sud
    'mx': 'mexico',           # Abbreviazione per il Messico
    'se': 'sweden',           # Abbreviazione per la Svezia
    'no': 'norway',           # Abbreviazione per la Norvegia
    'fi': 'finland',          # Abbreviazione per la Finlandia
    'nl': 'netherlands',      # Abbreviazione per i Paesi Bassi
    'ch': 'switzerland',      # Abbreviazione per la Svizzera
    'ru': 'russia',           # Abbreviazione per la Russia
    'sa': 'south africa',     # Abbreviazione per il Sudafrica
    'kr': 'south korea',      # Abbreviazione per la Corea del Sud
    'sg': 'singapore'         # Abbreviazione per Singapore
}

In [75]:
import re

# Nuovo dizionario di sostituzione per i codici fiscali italiani
CODE_REPLACEMENTS = {
    r"^it\d{10,16}$": "italy"  # Codici fiscali italiani che iniziano con "it" seguito da 10-16 numeri
}

def normalize_country(country):
    if pd.isna(country) or not isinstance(country, str):
        return ""
    
    country = country.strip().lower()
    
    # Se il valore è un codice fiscale italiano, lo sostituisce direttamente
    for pattern, replacement in CODE_REPLACEMENTS.items():
        if re.match(pattern, country):
            return replacement  
    
    # Se ci sono più paesi separati da virgole, li normalizziamo singolarmente
    country_list = [c.strip() for c in country.split(",")]  # Divide e rimuove spazi
    normalized_list = [country_mapping.get(c, c) for c in country_list]  # Applica il mapping
    
    return ", ".join(normalized_list)  # Ritorna la stringa normalizzata

# Applicare la normalizzazione della colonna 'country'
filtered_df['normalized_country'] = filtered_df['country'].apply(normalize_country)

# Visualizzare una selezione casuale delle righe per verificare
display(filtered_df[['company_name', 'country', 'normalized_country']].sample(25))


,company_name,country,normalized_country
71857,visage technologies,sweden,sweden
74067,xpand,portugal,portugal
17552,dalian huarui heavy industry group,china,china
1013,ad,south korea,south korea
51140,pridel,india,india
60165,skydance media,usa,united states
9258,ben lai,china,china
70003,utopia lab,italy,italy
68585,two harbors investment,usa,united states
72419,wafd bank,usa,united states


In [25]:
filtered_df[filtered_df['company_name'] == "international consolidated airlines group"]



,company_id,company_name,trade_name,industry,sector,categories,company_status,company_type,headquarters,address,...,facebook,twitter,pinterest,instagram,investors,region,notes_or_description,normalized_name,modified,normalized_country
33431,faf52c4c18e746ed853f699c2bcb1568,international consolidated airlines group,NaN,NaN,NaN,"['Industries', 'Airlines', 'Aviation', 'Transp...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,international consolidated airlines group,False,"united kingdom, spain"


,company_name,country
236,888,"['Gibralter', 'UK']"
4605,apax global alpha limited,"['UK', 'Guernsey']"
10330,bmo commercial property trust limited,"['UK', 'Guernsey']"
26681,globalworth real estate investments limited,"['UK', 'Guernsey']"
28613,gvc,"['Isle of Man', 'UK']"
29383,harbourvest global private equity limited,"['UK', 'Guernsey']"
30420,hipgnosis songs fund limited,"['UK', 'Guernsey']"
33431,international consolidated airlines group,"['UK', 'Spain']"
33478,international public partnerships ld,"['UK', 'Guernsey']"
35810,kape technologies,"['Isle of Man', 'UK']"


Eliminazione duplicati nello schema mediato nel file aziende_normalizzate.csv


In [83]:
import pandas as pd


aziende_df = pd.read_csv("../aziende_normalizzate.csv", encoding="utf-8")


# Conta i duplicati nel file delle aziende
aziende_duplicati = aziende_df.duplicated().sum()

# Trova le righe duplicate
aziende_duplicati_righe = aziende_df[aziende_df.duplicated()]

# Stampa i risultati
print(f"Numero di righe duplicate nel file 'aziende_normalizzate.csv': {aziende_duplicati}")

# Mostra le prime righe duplicate (se ci sono)
if aziende_duplicati > 0:
    print("\nEsempio di righe duplicate nel file 'aziende_normalizzate.csv':")
    print(aziende_duplicati_righe.head())
else:
    print("\nNon ci sono righe duplicate nel file.")


C:\Users\emigi\AppData\Local\Temp\ipykernel_27024\3067150653.py:4: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  aziende_df = pd.read_csv("../aziende_normalizzate.csv", encoding="utf-8")


Numero di righe duplicate nel file 'aziende_normalizzate.csv': 1482

Esempio di righe duplicate nel file 'aziende_normalizzate.csv':
     company_id             company_name trade_name  industry sector  \
1019   02921121  'q' accountancy limited        NaN       NaN    NaN   
1210   12816840         5d uk operations        NaN       NaN    NaN   
1212   12816020        5d uk real estate        NaN       NaN    NaN   
1226        NaN          6d technologies        NaN   telecom    NaN   
1268        NaN                  99acres        NaN  internet    NaN   

     categories company_status             company_type  \
1019        NaN         active  private limited company   
1210        NaN         active  private limited company   
1212        NaN         active  private limited company   
1226        NaN            NaN                      NaN   
1268        NaN            NaN                   public   

                               headquarters  \
1019                            

In [84]:
# Elimina i duplicati
aziende_df_senza_duplicati = aziende_df.drop_duplicates()

# Salva il DataFrame senza duplicati in un nuovo file CSV
aziende_df_senza_duplicati.to_csv("aziende_senza_duplicati.csv", index=False)

# Visualizza un'anteprima dei dati senza duplicati
print(aziende_df_senza_duplicati.head())

  company_id company_name trade_name  \
0       1470          NaN        NaN   
1          7          NaN        NaN   
2          8          NaN        NaN   
3          9          NaN        NaN   
4         11          NaN        NaN   

                                            industry sector categories  \
0   other business support service activities n.e.c.    NaN        NaN   
1      manufacture of plastic packing goods \xc2\xa0    NaN        NaN   
2  other retail sale not in stores, stalls or mar...    NaN        NaN   
3  activities of saunas, sunbeds and massage salo...    NaN        NaN   
4  business and other management consultancy acti...    NaN        NaN   

  company_status company_type headquarters address  ... nace_code facebook  \
0            NaN          NaN          NaN     NaN  ...     82.99      NaN   
1            NaN          NaN          NaN     NaN  ...     22.22      NaN   
2            NaN          NaN          NaN     NaN  ...     47.99      NaN   
3 